In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [3]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [30]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [31]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [32]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [33]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [34]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [39]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [36]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [57]:
stream_brochure("RoboVitics", "https://robovitics.in/")

Found links: {'relevant_links': ['https://robovitics.in/about-page', 'https://robovitics.in/events-page', 'https://robovitics.in/projects-page', 'https://robovitics.in/orcus-page', 'https://robovitics.in/kartavya-page', 'https://robovitics.in/sponsors-page', 'https://robovitics.in/memories', 'https://twitter.com/robo_vit', 'https://www.facebook.com/robovitics', 'https://www.instagram.com/robovitics/', 'https://www.linkedin.com/company/robovitics/', 'https://www.youtube.com/channel/UCFiwOI-W5b06NweratR-RdA', 'https://github.com/RoboVITics']}



# Welcome to RoboVitics: Where Robotics Dreams Come True!

> “A place to learn, a chance to grow” 

Get ready to immerse yourself into the world of tech, robots, and a generous sprinkle of imagination at RoboVitics! At **Vellore Institute of Technology**, we bring robotics enthusiasts together like ketchup and fries—inseparable!

## Who We Are

Founded as a small group of tech-lover rebels, we have blossomed into the **official Robotics Club of VIT**. Our mission? To spread our neural net of knowledge about robotics through free workshops, seminars, and the occasional pizza party. 

Innovating since 1984, our university was established by G. Viswanathan with the ambition to improve students' lives (and their caffeine intake) through stellar education and research.

### Club Culture

- **Passion Over Perfection**: We celebrate creativity, failures, and robot malfunctions! 
- **Learning is Fun**: Because who said you can't learn while battling robots? 
- **Collaboration Central**: Teaming up to build the coolest bot is our jam. Turns out, friendships built over robots last longer than the robots themselves.

## Events That Rock!

### 🎉 **EQUINOX**
A whirlwind of ideas, creativity, and sheer tech madness! Participate in a 36-hour extravaganza where your brain gets a workout while being mentored by some ridiculously knowledgeable folks.

### 🤖 **RoboWars**
Witness battle bots clash in an electrifying competition! It's like watching gladiators, but with machines and fewer lions. May the best robot win!

### ✏️ **Vortex360**
Our CAD modeling hackathon powered by AutoDesk—where designers let their creative juices flow faster than soda on a hot day.

### 🔧 **Hands On Robotics**
Join our annual workshop to design fully functional mobile Arduino Robots, because who hasn’t dreamed of controlling a robot army?

## Join Our Quest!

### Careers and Opportunities
Are you a tech enthusiast with a knack for battery-powered, circuit-filled mayhem? Join us! As a member of the RoboVIT team, you'll have the chance to learn, grow, and possibly build some innovative robots that may or may not take over the world (no promises)!

### Who We’re Looking For:
- Tech Wizards (ML & AI, Cybersecurity, Web & App Developers)
- Creative Designers (CAD Experts trying to let designs take flight)
- Those with aspirations, electric dreams, or just a passion for kicking metal butt!

## Connect With Us!
Don't be shy! Follow us on our social networks to stay updated on our techy antics and events—maybe we can brainstorm a new robot together (or just chat about pizza). 

- [Facebook](https://www.facebook.com/RoboVitics)
- [Instagram](https://www.instagram.com/RoboVitics)
- [Twitter](https://www.twitter.com/RoboVitics)

---

RoboVitics: Because it’s not just about robotics, it’s about creating fun memories and effective teamwork—one robot at a time! 🤖✨

